In [3]:
import asyncio
import nest_asyncio
from bleak import BleakScanner

nest_asyncio.apply()

async def main():
    async with BleakScanner() as scanner:
        devices = await scanner.discover()
        for d in devices:
            print(d)

if __name__ == '__main__':
    asyncio.run(main())

20:BF:CC:17:18:9B: None
1A:4A:40:10:64:11: None
02:B6:45:0B:36:04: None
FD:02:9A:E0:EA:AB: None
78:F6:89:C5:27:2E: None


In [ ]:
import asyncio
import nest_asyncio
from bleak import BleakClient

nest_asyncio.apply()

async def main(address):
    async with BleakClient(address, timeout=30) as client:
        if(not client.is_connected):
            raise 'Client not connected'
        
        for service in client.services:
            print('\nServices', service.handle, service.uuid, service.description)

            characteristcs = service.characteristics

            for char in characteristcs:
                print(' Characteristic', char.handle, char.uuid, char.description, char.properties)

                descriptors = char.descriptors

                for desc in descriptors:
                    print('   Descriptor', desc)

if __name__ == '__main__':
    address = 'C4:8C:B3:08:58:A3'
    asyncio.run(main(address), debug=True)

In [ ]:
import asyncio
import nest_asyncio
from bleak import BleakClient

nest_asyncio.apply()

async def notification_handler(sender: int, data: bytearray):
    decoded_data = data.decode('utf-8')
    print(f"Measured value: {decoded_data}")

async def monitor_data(address):
    async with BleakClient(address) as client:
        characteristic_uuid = "6e400003-b5a3-f393-e0a9-e50e24dcca9e"
        await client.start_notify(characteristic_uuid, notification_handler)

        try:
            while True:
                # The loop will be triggered when a notification is received
                await asyncio.sleep(1)  # Adjust the sleep interval as needed

        except Exception as e:
            print(f"Error monitoring data: {e}")

if __name__ == "__main__":
    device_address = "C4:8C:B3:08:58:A3"
    
    # Run the event loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(monitor_data(device_address))